## Problem Statement 

You need to build a model that is able to classify customer complaints based on the products/services. By doing so, you can segregate these tickets into their relevant categories and, therefore, help in the quick resolution of the issue.

You will be doing topic modelling on the <b>.json</b> data provided by the company. Since this data is not labelled, you need to apply NMF to analyse patterns and classify tickets into the following five clusters based on their products/services:

* Credit card / Prepaid card

* Bank account services

* Theft/Dispute reporting

* Mortgages/loans

* Others 


With the help of topic modelling, you will be able to map each ticket onto its respective department/category. You can then use this data to train any supervised model such as logistic regression, decision tree or random forest. Using this trained model, you can classify any new customer complaint support ticket into its relevant department.

## Pipelines that needs to be performed:

You need to perform the following eight major tasks to complete the assignment:

1.  Data loading

2. Text preprocessing

3. Exploratory data analysis (EDA)

4. Feature extraction

5. Topic modelling 

6. Model building using supervised learning

7. Model training and evaluation

8. Model inference

## Importing the necessary libraries

In [ ]:
import json 
import numpy as np
import pandas as pd
import re, nltk, spacy, string
import en_core_web_sm
nlp = en_core_web_sm.load()
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from plotly.offline import plot
import plotly.graph_objects as go
import plotly.express as px

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from pprint import pprint

## Loading the data

The data is in JSON format and we need to convert it to a dataframe.

In [ ]:
# Setting max rows and columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# Opening JSON file 
f = open("complaints-2021-05-14_08_16.json") # Write the path to your data file and load it 
  
# returns JSON object as  
# a dictionary 
data = json.load(f)
df=pd.json_normalize(data)

## Data preparation

In [ ]:
# Inspect the dataframe to understand the given data.
df.info()

In [ ]:
#print the column names
pprint(df.columns)

In [ ]:
#Assign new column names
df.columns = ['index', 'type', 'id', 'score', 'tags', 'zip_code','complaint_id', 'issue', 'date_received',
       'state', 'consumer_disputed', 'product','company_response', 'company', 'submitted_via',
       'date_sent_to_company', 'company_public_response','sub_product', 'timely',
       'complaint_what_happened', 'sub_issue','consumer_consent_provided']

In [ ]:
#Assign nan in place of blanks in the complaints column
df[df.loc[:, 'complaint_what_happened'] == ''] = np.nan

In [ ]:
#Remove all rows where complaints column is nan
df = df[~df['complaint_what_happened'].isnull()]

## Prepare the text for topic modeling

Once you have removed all the blank complaints, you need to:

* Make the text lowercase
* Remove text in square brackets
* Remove punctuation
* Remove words containing numbers


Once you have done these cleaning operations you need to perform the following:
* Lemmatize the texts
* Extract the POS tags of the lemmatized text and remove all the words which have tags other than NN[tag == "NN"].


In [ ]:
# Write your function here to clean the text and remove all the unnecessary elements.
def clean_text(sentence):
    sentence = sentence.lower() # Text to lowercase
    sentence = re.sub('[^\w\s]', '', sentence) # Remove punctuation
    sentence = re.sub('\w*\d\w*', '', sentence) # Remove words containing numbers
    sentence = re.sub('\[.*?\]', '', sentence) # Remove text in square brackets
    return sentence

In [ ]:
df_clean = pd.DataFrame(df['complaint_what_happened'].apply(clean_text))

In [ ]:
#Write your function to Lemmatize the texts
def lemmmatize_text(text):
    lemm_list = []
    doc = nlp(text)
    for token in doc:
        lemm_list.append(token.lemma_)
    return " ".join(lemm_list)

In [ ]:
#Create a dataframe('df_clean') that will have only the complaints and the lemmatized complaints 
df_clean['complaint_lemmatized'] = df_clean['complaint_what_happened'].apply(lemmmatize_text)

In [ ]:
df_clean

In [ ]:
#Write your function to extract the POS tags 
from textblob import TextBlob

def pos_tag(text):
  # write your code here
    sent = []
    blob = TextBlob(text)
    sent = [word for (word,tag) in blob.tags if tag=='NN']
    return " ".join(sent)

In [ ]:
#this column should contain lemmatized text with all the words removed which have tags other than NN[tag == "NN"].
df_clean["complaint_POS_removed"] = df_clean['complaint_lemmatized'].apply(pos_tag)

In [ ]:
#The clean dataframe should now contain the raw complaint, lemmatized complaint and the complaint after removing POS tags.
df_clean

## Exploratory data analysis to get familiar with the data.

Write the code in this task to perform the following:

*   Visualise the data according to the 'Complaint' character length
*   Using a word cloud find the top 40 words by frequency among all the articles after processing the text
*   Find the top unigrams,bigrams and trigrams by frequency among all the complaints after processing the text. ‘




In [ ]:
# Write your code here to visualise the data according to the 'Complaint' character length
char_len = [len(each_sent) for each_sent in df_clean['complaint_POS_removed']]

sns.displot(char_len, kind='hist', bins=50)
plt.ylabel("Total no of Complaints")
plt.xlabel("Complaint Character Length")
plt.title("Distribution of Complaint Character Length")
plt.show()

#### Find the top 40 words by frequency among all the articles after processing the text.

In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [ ]:
#Using a word cloud find the top 40 words by frequency among all the articles after processing the text
stopwords = set(STOPWORDS)
wordcloud = WordCloud( background_color="white",
                       max_font_size=60, 
                       max_words=40, 
                       random_state=100, 
                    stopwords=stopwords).generate(str(df_clean['complaint_POS_removed']))
plt.figure(figsize=[15,12])
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

- Most frequent words are Credit,card,chase,bank

In [ ]:
#Removing -PRON- from the text corpus
df_clean['Complaint_clean'] = df_clean['complaint_POS_removed'].str.replace('-PRON-', '')

#### Find the top unigrams,bigrams and trigrams by frequency among all the complaints after processing the text.

In [ ]:
#Write your code here to find the top 30 unigram frequency among the complaints in the cleaned datafram(df_clean). 
def get_top_ngrams(text, n=30,ngram=(1,1)):
      vector = CountVectorizer(stop_words='english',ngram_range=ngram).fit(text)
      bagofwords = vector.transform(text)
      sum_words = bagofwords.sum(axis=0)
      words_frequency = [(word, sum_words[0, index]) for word, index in vector.vocabulary_.items()]
      words_frequency = sorted(words_frequency, key = lambda x: x[1], reverse=True)
      return words_frequency[:n]

In [ ]:
#Print the top 10 words in the unigram frequency
top_words = get_top_ngrams(df_clean['Complaint_clean'].values.astype('U'),ngram=(1,1))
df_unigram = pd.DataFrame(top_words, columns=['unigram', 'count'])
df_unigram.head(10)

In [ ]:
#Write your code here to find the top 30 bigram frequency among the complaints in the cleaned datafram(df_clean). 
top_words = get_top_ngrams(df_clean['Complaint_clean'].values.astype('U'), ngram=(2,2))
df_bigram = pd.DataFrame(top_words, columns=['bigram', 'count'])
df_bigram

In [ ]:
#Print the top 10 words in the bigram frequency
df_bigram.head(10)

In [ ]:
#Write your code here to find the top 30 trigram frequency among the complaints in the cleaned datafram(df_clean). 
top_words = get_top_ngrams(df_clean['Complaint_clean'].values.astype('U'), ngram=(3,3))
df_trigram = pd.DataFrame(top_words, columns=['trigram', 'count'])
df_trigram

In [ ]:
#Print the top 10 words in the trigram frequency
df_trigram.head(10)

## The personal details of customer has been masked in the dataset with xxxx. Let's remove the masked text as this will be of no use for our analysis

In [ ]:
df_clean['Complaint_clean'] = df_clean['Complaint_clean'].str.replace('xxxx','')

In [ ]:
#All masked texts has been removed
df_clean.shape

## Feature Extraction
Convert the raw texts to a matrix of TF-IDF features

**max_df** is used for removing terms that appear too frequently, also known as "corpus-specific stop words"
max_df = 0.95 means "ignore terms that appear in more than 95% of the complaints"

**min_df** is used for removing terms that appear too infrequently
min_df = 2 means "ignore terms that appear in less than 2 complaints"

In [ ]:
#Write your code here to initialise the TfidfVectorizer 
tfidf = TfidfVectorizer(min_df=2, max_df=0.95, stop_words='english')


#### Create a document term matrix using fit_transform

The contents of a document term matrix are tuples of (complaint_id,token_id) tf-idf score:
The tuples that are not there have a tf-idf score of 0

In [ ]:
#Write your code here to create the Document Term Matrix by transforming the complaints column present in df_clean.
dtm = tfidf.fit_transform(df_clean['Complaint_clean'])
dtm

## Topic Modelling using NMF

Non-Negative Matrix Factorization (NMF) is an unsupervised technique so there are no labeling of topics that the model will be trained on. The way it works is that, NMF decomposes (or factorizes) high-dimensional vectors into a lower-dimensional representation. These lower-dimensional vectors are non-negative which also means their coefficients are non-negative.

In this task you have to perform the following:

* Find the best number of clusters 
* Apply the best number to create word clusters
* Inspect & validate the correction of each cluster wrt the complaints 
* Correct the labels if needed 
* Map the clusters to topics/cluster names

In [ ]:
from sklearn.decomposition import NMF

## Manual Topic Modeling
You need to do take the trial & error approach to find the best num of topics for your NMF model.

The only parameter that is required is the number of components i.e. the number of topics we want. This is the most crucial step in the whole topic modeling process and will greatly affect how good your final topics are.

In [ ]:
#Load your nmf_model with the n_components i.e 5
num_topics = 5 
#write the value you want to test out

#keep the random_state =40
nmf_model = NMF(n_components=num_topics, random_state=40)

In [ ]:
nmf_model.fit(dtm)
len(tfidf.get_feature_names())

In [ ]:
#Print the Top15 words for each of the topics
H = nmf_model.components_ 
words = np.array(tfidf.get_feature_names())
topic_words = pd.DataFrame(np.zeros((num_topics, 15)), index=[f'Topic {i + 1}' for i in range(num_topics)],
                           columns=[f'Word {i + 1}' for i in range(15)]).astype(str)
for i in range(num_topics):
    ix = H[i].argsort()[::-1][:15]
    topic_words.iloc[i] = words[ix]

topic_words

#### Findings:
- Topic 1 = Bank account services
- Topic 2 = Credit card / Prepaid card
- Topic 3 = Others
- Topic 4 = Theft/Dispute reporting
- Topic 5 = Mortgages/loans

In [ ]:
#Create the best topic for each complaint in terms of integer value 0,1,2,3 & 4
topic_results = nmf_model.transform(dtm)


In [ ]:
#Assign the best topic to each of the cmplaints in Topic Column

df_clean['Topic'] = topic_results.argmax(axis=1)

In [ ]:
df_clean.shape

In [ ]:
#Print the first 5 Complaint for each of the Topics
df_clean_print=df_clean.groupby('Topic').head(5)
df_clean_print.sort_values('Topic')

#### After evaluating the mapping, if the topics assigned are correct then assign these names to the relevant topic:
* Bank Account services
* Credit card or prepaid card
* Theft/Dispute Reporting
* Mortgage/Loan
* Others

In [ ]:
#Create the dictionary of Topic names and Topics

Topic_names = { 0:"Bank account services", 1:"Credit card / Prepaid card", 2:"Others",
               3:"Theft/Dispute reporting", 4:"Mortgages/loans" }
#Replace Topics with Topic Names
df_clean['Topic'] = df_clean['Topic'].map(Topic_names)

In [ ]:
df_clean

## Supervised model to predict any new complaints to the relevant Topics.

You have now build the model to create the topics for each complaints.Now in the below section you will use them to classify any new complaints.

Since you will be using supervised learning technique we have to convert the topic names to numbers(numpy arrays only understand numbers)

In [ ]:
#Create the dictionary again of Topic names and Topics

Topic_names = { "Bank account services":0, "Credit card / Prepaid card":1, "Others":2,
               "Theft/Dispute reporting":3, "Mortgages/loans":4 }
#Replace Topics with Topic Names
df_clean['Topic'] = df_clean['Topic'].map(Topic_names)

In [ ]:
#Keep the columns"complaint_what_happened" & "Topic" only in the new dataframe --> training_data
training_data=df_clean[['complaint_what_happened', 'Topic']]

In [ ]:
training_data

#### Apply the supervised models on the training data created. In this process, you have to do the following:
* Create the vector counts using Count Vectoriser
* Transform the word vecotr to tf-idf
* Create the train & test data using the train_test_split on the tf-idf & topics


In [ ]:
import pickle

In [ ]:
#Write your code to get the Vector count
vector_count = CountVectorizer()
X_train_count = vector_count.fit_transform(training_data['complaint_what_happened'])
pickle.dump(vector_count.vocabulary_, open("count_vector.pk1", "wb"))

#Write your code here to transform the word vector to tf-idf
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_count)
pickle.dump(tfidf_transformer, open('tfidf.pk1', "wb"))

You have to try atleast 3 models on the train & test data from these options:
* Logistic regression
* Decision Tree
* Random Forest
* Naive Bayes (optional)

**Using the required evaluation metrics judge the tried models and select the ones performing the best**

# Model1 - Logistic Regression

In [ ]:
# Importing packages
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


In [ ]:
# Split data into train and test
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, training_data['Topic'], test_size=0.25, random_state=42)

In [ ]:
# creating model
lr = LogisticRegression().fit(X_train, y_train)

In [ ]:
# checking score
lr.score(X_test, y_test)

##### HyperParameter Tuning

In [ ]:
# setting parameters
lr_param = {"C": [4, 0.1, 0.01,0.001,10,2,1,4,5,3],
            'penalty': ['l1', 'l2'],
                'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}

In [ ]:
# fitting model with hyperparameter tuning
from sklearn.model_selection import GridSearchCV
lr_tuned = GridSearchCV(estimator=lr,
                                param_grid=lr_param,
                                cv=4,
                                verbose=True,
                                n_jobs=-1)

lr_tuned.fit(X_train, y_train);

In [ ]:
# best parameters
lr_tuned.best_params_

In [ ]:
# score after using hyper parameter
lr_tuned.score(X_test, y_test)

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report,f1_score

In [ ]:
# saving model
pickle.dump(lr_tuned, open("logreg_model.pk1", "wb"))

In [ ]:
# loading model
logreg_model = pickle.load(open("logreg_model.pk1", "rb"))

In [ ]:
# predicting y
y_pred_lr = logreg_model.predict(X_test)

# model evaluation

In [ ]:
# confusion matrix
print(confusion_matrix(y_test, y_pred_lr))

In [ ]:
# confusion matrix in heatmap visualization
fig, ax = plt.subplots(figsize=(6, 6))
ax = sns.heatmap(confusion_matrix(y_test, y_pred_lr),
                 annot=True,
                 cbar=False,
                 cmap="RdYlGn", fmt = '0.1f')
plt.xlabel("Actual label")
plt.ylabel("Predicted label")
plt.show()

In [ ]:
Topicnames_target = ["Bank account services", "Credit card / Prepaid card", "Others", "Theft/Dispute reporting", "Mortgages/loans"]

In [ ]:
print(classification_report(y_test, y_pred_lr, target_names = Topicnames_target))

### Findings
- the accuracy is 94%, which is a good model with good precision and recall value

# Model 2 - Decision Tree

In [ ]:
# importing packages
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# splitting dataset
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, training_data['Topic'], test_size=0.25, random_state=42)

In [ ]:
# decision tree model with default parameter
dt = DecisionTreeClassifier().fit(X_train, y_train)

In [ ]:
# score
dt.score(X_test, y_test)

#### Hyperparameter tuning

In [ ]:
# setting parameters
dt_grid = {"max_depth": [3,5,7,9],
           "min_samples_split": [5,10,15,20,25,30],
           "min_samples_leaf": [1,5,10,15,20,25]}

In [ ]:
# fitting model
dt_tuned = GridSearchCV(estimator=dt,
                                param_grid=dt_grid,
                                cv=5,
                                verbose=True,
                                n_jobs=-1)

dt_tuned.fit(X_train, y_train);

In [ ]:
# best parameters
dt_tuned.best_params_

In [ ]:
# score after tuning
dt_tuned.score(X_test, y_test)

- accuracy is bit lower than basic model

In [ ]:
# Save decision tree model
pickle.dump(dt_tuned, open("dt_model.pk1", "wb"))

In [ ]:
# Load model
dt_model = pickle.load(open("dt_model.pk1", "rb"))

In [ ]:
# predictions on test data
y_pred = dt_model.predict(X_test)

## model Evaluation

In [ ]:
# Confusion Matrix
print(confusion_matrix(y_test, y_pred))

In [ ]:
# confusion matrix in heatmap visualization
fig, ax = plt.subplots(figsize=(6, 6))
ax = sns.heatmap(confusion_matrix(y_test, y_pred),
                 annot=True,
                 cbar=False,
                 cmap="RdYlGn", fmt = '0.1f')
plt.xlabel("Actual label")
plt.ylabel("Predicted label")
plt.show()

In [ ]:
Topicnames_target = ["Bank account services", "Credit card / Prepaid card", "Others", "Theft/Dispute reporting", "Mortgages/loans"]

In [ ]:
# Classification Report
print(classification_report(y_test, y_pred, target_names = Topicnames_target))

## Findings
- accuracy is 77% which is lower than the logistic model

# Random Forest

In [ ]:
# importing packages
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# split data into train and test
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, training_data['Topic'], test_size=0.25, random_state=42)

In [ ]:
# fitting the basic model
rf = RandomForestClassifier().fit(X_train, y_train)

In [ ]:
#Score of default model
rf.score(X_test, y_test)

# HyperParameter Tuning

In [ ]:
# setting hyper parameters
rf_grid = {"n_estimators": [50, 100, 250],
           "max_depth": [5,10,15],
           "min_samples_split": [10,15,20],
           "min_samples_leaf": [5,10,15]}

In [ ]:
# fitting model with hypermeters
rf_tuned = GridSearchCV(estimator=rf,
                                param_grid=rf_grid,
                                cv=5,
                                verbose=True,
                                n_jobs=-1)

rf_tuned.fit(X_train, y_train);

In [ ]:
# best parameters
rf_tuned.best_params_

In [ ]:
# score after hyper tuning
rf_tuned.score(X_test, y_test)

- accuracy is lower than the basic model

In [ ]:
# Save Model
pickle.dump(rf_tuned, open("rf_model.pk1", "wb"))

In [ ]:
# Load model
rf_model = pickle.load(open("rf_model.pk1", "rb"))

In [ ]:
# predictions on test data
y_pred = rf_model.predict(X_test)

## model evaluation

In [ ]:
# Confusion Matrix
print(confusion_matrix(y_test, y_pred))

In [ ]:
# confusion matrix in heatmap visualization
fig, ax = plt.subplots(figsize=(6, 6))
ax = sns.heatmap(confusion_matrix(y_test, y_pred),
                 annot=True,
                 cbar=False,
                 cmap="RdYlGn", fmt = '0.1f')
plt.xlabel("Actual label")
plt.ylabel("Predicted label")
plt.show()

In [ ]:
Topicnames_target = ["Bank account services", "Credit card / Prepaid card", "Others", "Theft/Dispute reporting", "Mortgages/loans"]

In [ ]:
# Classification Report
print(classification_report(y_test, y_pred, target_names = Topicnames_target))

### findings
- Overall accuracy is 75, which is lower than logistic model

# conclusion
- we created 3 models using Logistic Regression, Decision Tree and Random Forest, out of this Logistic regression model performed well with anaccuracy of 94%